# Classifictaion Task

### Goals
Our goal to predict whether each costumer will be repurchase 
or not within 14 days.

### Steps
* Import Libraries 
* Collecting Data
* Preparing Data
* Feature Engineering
* Train Model
* Predict
* Conclusion




### Introduction
In this notebook I will show you the solving of a classification problem. We have 2 years 
worth of data. We need to predict last 14 days worth of repurchase counts and the customers
that will be repurchasing. We will implement some methods there and will compare how good are they;
<br><br><br>
### 1. Import Libraries 
First of all lets import libraries which will be need in further coding.

In [1]:
from __future__ import division

# import libraries
from datetime import date
import pandas as pd
from sklearn.cluster import KMeans

# do not show warnings
import warnings

warnings.filterwarnings("ignore")


# import machine learning related libraries
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb

from database.database import Database, schema_name, invoice_table, invoice_product_table, product_table

### 2. Collecting Data
Now lets collect the data on which we will be working. We are using database
for storing data. So we need to create database object and select query.

In [2]:
database = Database('../../database/database.ini')

#
query = f"""select it.invoice_date, it.customer_id, ipt.quantity, pt.price
            from {schema_name}.{invoice_table} it
                inner join {schema_name}.{invoice_product_table} ipt 
                    on it.invoice_no = ipt.invoice_no
                inner join {schema_name}.{product_table} pt 
                    on ipt.stock_code = pt.stock_code
            where it.customer_id <> 'NaN' and
					left(it.invoice_no,1)<>'C'
            order by it.invoice_date  """

Everything is ready, now we can collect that data from database with the select function
we create in Database Class; Because that function returns list we will turn that list 
into Dataframe with the help of pandas.

In [3]:
select = database.select(query)
data = pd.DataFrame(select, columns=['InvoiceDate', 'Customer ID', 'Quantity', 'Price'])
data.head(10)

,InvoiceDate,Customer ID,Quantity,Price
0,2009-12-01,17063,1,2.25
1,2009-12-01,17063,8,4.95
2,2009-12-01,17063,10,3.75
3,2009-12-01,17063,3,0.00
4,2009-12-01,17063,2,2.95
5,2009-12-01,17063,2,2.95
6,2009-12-01,17063,2,2.95
7,2009-12-01,15719,2,2.51
8,2009-12-01,15719,2,2.51
9,2009-12-01,15719,4,1.25


### 3. Preparing Data
Now we can start to prepare data for model with its features. 

As you can see we have 4 columns: Invoice Date, Customer Id, Quantity and Price.  
Now we will pick split our data in two parts. We will take last 6 month for prediction
and then other part for train.

In [4]:
train_date_before = data[data['InvoiceDate'] < date(2011, 11, 25)].reset_index(drop=True)
train_date_after = data[data['InvoiceDate'] >= date(2011, 11, 25)].reset_index(drop=True)

Also, we will create a dataframe called customers to possess a user-level 
feature set for the prediction model:

In [5]:
customers = pd.DataFrame(train_date_before['Customer ID'].unique())
customers.columns = ['Customer ID'] 

By using the data in train_date_after, we need the calculate our label 
(days between last purchase before cut off date and first purchase after that):

In [6]:
# Create a Dataframe with Custoemr ID and First Purchase date after given Date
customer_next_purchase = train_date_after.groupby('Customer ID').InvoiceDate.min().reset_index()
customer_next_purchase.columns = ['Customer ID', 'MinPurchaseDate']

# Create a Dataframe with Customer ID and last Purchase Date before given Date
customer_last_purchase = train_date_before.groupby('Customer ID').InvoiceDate.max().reset_index()
customer_last_purchase.columns = ['Customer ID', 'MaxPurchaseDate']

 # Merge two Dataframes
purchase_dates = pd.merge(customer_last_purchase, customer_next_purchase, on='Customer ID', how='left')

# Calculate the time difference in days
purchase_dates['NextPurchaseDay'] = (
    purchase_dates['MinPurchaseDate'] - purchase_dates['MaxPurchaseDate']).dt.days

# Merge with Customers
customers = pd.merge(customers, purchase_dates[['Customer ID', 'NextPurchaseDay']], 
                     on='Customer ID', how='left')

# Fill NA values with 999
customers = customers.fillna(999)

customers.head()

,Customer ID,NextPurchaseDay
0,17063,999.0
1,15719,999.0
2,12490,12.0
3,15793,25.0
4,17056,999.0


As you can see, we have NaN values because those customers haven’t made any purchase yet. We fill NaN with 999 
to quickly identify them later.

We have customer ids and corresponding labels in a dataframe. 
Let’s enrich it with our feature set to build our machine learning model.

### 4. Feature Engineering

For this project, we have selected our feature candidates like below:
* RFM scores & clusters
* Days between the last three purchases
* Mean & standard deviation of the difference between purchases in days

After adding these features, we need to deal with the categorical features by applying **get_dummies** method.
For **RFM** the code would be like

In [7]:
# Max Purchase date for Recency
max_purchase = train_date_before.groupby('Customer ID').InvoiceDate.max().reset_index()
max_purchase.columns = ['Customer ID', 'MaxPurchaseDate']

# Find the recency in days and add it to Customers
max_purchase['Recency'] = (max_purchase['MaxPurchaseDate'].max() - max_purchase['MaxPurchaseDate']).dt.days
customers = pd.merge(customers, max_purchase[['Customer ID', 'Recency']], on='Customer ID')

def order_cluster(cluster_field_name, target_field_name, dataframe, ascending):
    new_cluster_field_name = 'new_' + cluster_field_name

    datafarme_new = dataframe.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    datafarme_new = datafarme_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)
    datafarme_new['index'] = datafarme_new.index
    dataframe_final = pd.merge(dataframe, datafarme_new[[cluster_field_name, 'index']], on=cluster_field_name)
    dataframe_final = dataframe_final.drop([cluster_field_name], axis=1)
    dataframe_final = dataframe_final.rename(columns={'index': cluster_field_name})
    return dataframe_final

# Clustering for Recency
kmeans = KMeans(n_clusters=4)
kmeans.fit(customers[['Recency']])
customers['RecencyCluster'] = kmeans.predict(customers[['Recency']])


# This order_cluster function is function created by me and
# it is implemented in the start of the file

# Order Recency Clusters
customers = order_cluster('RecencyCluster', 'Recency', customers, False)

# Print Cluster Characteristics
customers.groupby('RecencyCluster')['Recency'].describe()

,count,mean,std,min,25%,50%,75%,max
RecencyCluster,,,,,,,,
0,648.0,606.688272,64.040403,503.0,553.0,604.0,654.25,723.0
1,1143.0,396.585302,47.153041,300.0,364.0,393.0,423.50,501.0
2,1024.0,200.786133,52.155476,118.0,156.0,197.0,246.00,298.0
3,3009.0,33.722832,29.959130,0.0,9.0,24.0,52.00,116.0


In [8]:
# Get Total purchases for Frequency Scores
frequency = train_date_before.groupby('Customer ID').InvoiceDate.count().reset_index()
frequency.columns = ['Customer ID', 'Frequency']

# Add Frequency column to Customers
customers = pd.merge(customers, frequency, on='Customer ID')

# Clustering for Frequency

kmeans = KMeans(n_clusters=4)
kmeans.fit(customers[['Frequency']])
customers['FrequencyCluster'] = kmeans.predict(customers[['Frequency']])

# Order Frequency cluster and Show the Characteristics

customers = order_cluster('FrequencyCluster', 'Frequency', customers, True)
customers.groupby('FrequencyCluster')['Frequency'].describe()

# Calculate monetary value, Create a Dataframe With it
train_date_before['Revenue'] = train_date_before['Quantity'] * train_date_before['Price']
train_date_before['Revenue'] = pd.to_numeric(train_date_before['Revenue'])
revenue = train_date_before.groupby('Customer ID').Revenue.sum().reset_index()

customers = pd.merge(customers, revenue, on='Customer ID')

# Revenue Clusters
kmeans = KMeans(n_clusters=4)
kmeans.fit(customers[['Revenue']])
customers['RevenueCluster'] = kmeans.predict(customers[['Revenue']])

# Ordering Clusters and who the Characteristics
customers = order_cluster('RevenueCluster', 'Revenue', customers, True)
customers.groupby('RevenueCluster')['Revenue'].describe()

# Building overall Segmentation
customers['OverallScore'] = customers['RecencyCluster'] + customers['FrequencyCluster'] \
                                        + customers['RevenueCluster']

# Assign Segment Names
customers['Segment'] = 'Low-Value'
customers.loc[customers['OverallScore'] > 2, 'Segment'] = 'Mid-Value'
customers.loc[customers['OverallScore'] > 4, 'Segment'] = 'High-Value'

We will be using **shift()** method a lot in this part.
First, we create a dataframe with Customer ID and Invoice Day (not datetime).
Then we will remove the duplicates since customers can do multiple purchases 
in a day and difference will become 0 for those.

In [9]:
# Create a Datafram with Custoemr ID and Invoice Date
day_order = train_date_before[['Customer ID', 'InvoiceDate']]

# Convert Invoice Datetime to day

day_order['InvoiceDay'] = train_date_before['InvoiceDate']

day_order = day_order.sort_values(['Customer ID', 'InvoiceDate'])

# Drop Duplicates
day_order = day_order.drop_duplicates(subset=['Customer ID', 'InvoiceDay'], keep='first')

Now we will use shift method and we create new columns with the dates 
of last 3 purchases and see how our dataframe looks like:

In [10]:
# Shifting last 3 Purchase dates
day_order['PrevInvoiceDate'] = day_order.groupby('Customer ID')['InvoiceDay'].shift(1)
day_order['T2InvoiceDate'] = day_order.groupby('Customer ID')['InvoiceDay'].shift(2)
day_order['T3InvoiceDate'] = day_order.groupby('Customer ID')['InvoiceDay'].shift(3)

day_order.head()

,Customer ID,InvoiceDate,InvoiceDay,PrevInvoiceDate,T2InvoiceDate,T3InvoiceDate
21071,12346,2009-12-14,2009-12-14,NaN,NaN,NaN
28059,12346,2009-12-18,2009-12-18,2009-12-14,NaN,NaN
30762,12346,2010-01-04,2010-01-04,2009-12-18,2009-12-14,NaN
38060,12346,2010-01-14,2010-01-14,2010-01-04,2009-12-18,2009-12-14
43840,12346,2010-01-22,2010-01-22,2010-01-14,2010-01-04,2009-12-18


Now lets calculate the difference between days for each invoice date and see how it looks like:

In [11]:
# Find the Difference between days
day_order['DayDiff'] = (day_order['InvoiceDay'] - day_order['PrevInvoiceDate']).dt.days
day_order['DayDiff2'] = (day_order['InvoiceDay'] - day_order['T2InvoiceDate']).dt.days
day_order['DayDiff3'] = (day_order['InvoiceDay'] - day_order['T3InvoiceDate']).dt.days

day_order.head()

,Customer ID,InvoiceDate,InvoiceDay,PrevInvoiceDate,T2InvoiceDate,T3InvoiceDate,DayDiff,DayDiff2,DayDiff3
21071,12346,2009-12-14,2009-12-14,NaN,NaN,NaN,NaN,NaN,NaN
28059,12346,2009-12-18,2009-12-18,2009-12-14,NaN,NaN,4.0,NaN,NaN
30762,12346,2010-01-04,2010-01-04,2009-12-18,2009-12-14,NaN,17.0,21.0,NaN
38060,12346,2010-01-14,2010-01-14,2010-01-04,2009-12-18,2009-12-14,10.0,27.0,31.0
43840,12346,2010-01-22,2010-01-22,2010-01-14,2010-01-04,2009-12-18,8.0,18.0,35.0


For each customer ID, we utilize **.agg()** method to find out the mean and 
standard deviation of the difference between purchases in days:

In [12]:
# Find out the mean and standard devition of the diference between purchases in days

day_diff = day_order.groupby('Customer ID').agg({'DayDiff': ['mean', 'std']}).reset_index()
day_diff.columns = ['Customer ID', 'DayDiffMean', 'DayDiffStd']

Now we are going to make a tough decision. The calculation above is quite useful 
for customers who have many purchases. But we can’t say the same for the ones with 1–2 purchases. 
For instance, it is too early to tag a customer as frequent who has only 2 purchases but back to back.
We only keep customers who have > 3 purchases by using the following line:

In [13]:
day_order_last = day_order.drop_duplicates(subset=['Customer ID'], keep='last')

Finally, we drop NA values, merge new dataframes with customers and apply **.get_dummies()** 
for converting categorical values:

In [14]:
day_order_last = day_order_last.dropna()

day_order_last = pd.merge(day_order_last, day_diff, on='Customer ID')

customers = pd.merge(customers, day_order_last[
            ['Customer ID', 'DayDiff', 'DayDiff2', 'DayDiff3', 'DayDiffMean', 'DayDiffStd']], on='Customer ID')

# Create the copy of Customer Dataframe to apply get_dummies

customers_copy = customers.copy()
customers_copy = pd.get_dummies(customers_copy)

Now we can set model and train it. But the problem is which one is better for this data.
But before we choose the model lets identify the classes.

We will be having Two classes classes:
* 0–14: Customers that will purchase in first 14 days — Class name: 1
* ≥ 14: Customers that will purchase in more than 14 days — Class name: 0

In [15]:
customers_copy['NextPurchaseDayRange'] = 1
customers_copy.loc[customers_copy.NextPurchaseDay > 14, 'NextPurchaseDayRange'] = 0

### 5. Selecting Model
Now we have to select the best model for our data. For this we can test all models scores and pick
the best one among them.

In [16]:
# Train and Test Split
customers_copy = customers_copy.drop('NextPurchaseDay', axis=1)

X, y = customers_copy.drop('NextPurchaseDayRange', axis=1), customers_copy.NextPurchaseDayRange

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

# This is to measure models
# Create an array of models
models = []
models.append((('LR', LogisticRegression())))
models.append(('NB', GaussianNB()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVC', SVC()))
models.append(("Dtree", DecisionTreeClassifier()))
models.append(("XGB", xgb.XGBClassifier()))
models.append(("KNN", KNeighborsClassifier()))

# Measure the accuracy
for name, model in models:
    kfold = KFold(n_splits=2, random_state=22)
    result = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    print(name, result)
    

LR [0.94550959 0.93138244]
NB [0.86781029 0.90010091]
RF [0.94651867 0.93541877]
SVC [0.9283552  0.91826438]
Dtree [0.9394551  0.93440969]
XGB [0.95055499 0.94046418]
KNN [0.9172553  0.91624622]


### 6. Train Model
As you can see all of the models resulted more than 92% accuracy except the **Gaussian**,
so any model would be acceptable. lets pick **RandomForest** model and see the results.

In [17]:
rf_model = RandomForestClassifier().fit(X_train, y_train)

print('Accuracy of RF classifier on training set: {:.2f}'
              .format(rf_model.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.2f}'
              .format(rf_model.score(X_test[X_train.columns], y_test)))

Accuracy of RF classifier on training set: 1.00
Accuracy of RF classifier on test set: 0.93


### 7. Predict
As you can see it got good accuracy. Now the one thing is left and that is to predict and 
find out which custoemr will repurchase.

In [18]:
prediction = rf_model.predict(customers_copy.drop('NextPurchaseDayRange', axis=1))
# Customers who will repurchase within 14 days
repurchase_customers = []

for i in range(0, len(prediction)):
    if prediction[i] == 1:
        repurchase_customers.append(customers_copy.columns[i])
print('Customers Who will repurchase: ')
print("\n".join(repurchase_customers))


Customers Who will repurchase: 
Customer ID_12361
Customer ID_12395
Customer ID_12399
Customer ID_12407
Customer ID_12481
Customer ID_12504
Customer ID_12518
Customer ID_12577
Customer ID_12586
Customer ID_12600
Customer ID_12633
Customer ID_12648
Customer ID_12757
Customer ID_12830
Customer ID_12854
Customer ID_12889
Customer ID_12937
Customer ID_12948
Customer ID_13104
Customer ID_13173
Customer ID_13330
Customer ID_13362
Customer ID_13531
Customer ID_13569
Customer ID_13711
Customer ID_13770
Customer ID_13778
Customer ID_13798
Customer ID_14063
Customer ID_14189
Customer ID_14199
Customer ID_14249
Customer ID_14408
Customer ID_14493
Customer ID_14534
Customer ID_14549
Customer ID_14693
Customer ID_14738
Customer ID_14819
Customer ID_14829
Customer ID_14915
Customer ID_15003
Customer ID_15057
Customer ID_15061
Customer ID_15100
Customer ID_15125
Customer ID_15221
Customer ID_15304
Customer ID_15347
Customer ID_15359
Customer ID_15382
Customer ID_15410
Customer ID_15421
Customer ID_15

### 8. Conclusion
To sum up, this classification model was a very successful one. We prepared some features to add 
in the model and it turned out to be a good attempt. In regression tasks we used default parameters 
and we did not add any features and it could not reached a good result. With this classification
example we can say for sure that features will significantly improve the accuracy of the model.
